In [ ]:
import sys
sys.path.append("../")
import os

import xarray as xr
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from distributed import LocalCluster, Client

import src_screening.model.accessor
from src_screening.model.wave_forcing import WaveForcing

import matplotlib.pyplot as plt
import cmocean
import matplotlib.gridspec as mpl_gs

In [ ]:
plt.style.use("paper")
plt.style.use("egu_journals")

In [ ]:
local_cluster = LocalCluster(n_workers=48, local_directory="/tmp")
client = Client(local_cluster)

## Load data

In [ ]:
ds_hr_nature = xr.open_zarr("../../data/raw/test/hr_nature_forecast/")
ds_forecast = xr.open_zarr("../../data/raw/test/lr_forecast/")
ds_hybrid = xr.open_dataset(
    "../../data/processed/unext_small/9/traj_short/", engine="zarr",
    chunks={"time": -1, "ensemble": 1, "lead_time": -1, "nMesh2_node": -1, "nMesh2_face": -1}
)

# Create nice animation

In [ ]:
pltdata_hr = ds_hr_nature.sel(ensemble=41, time="1970-01-02T04:00:00")
pltdata_forecast = ds_forecast.sel(ensemble=41, time="1970-01-02T04:00:00")
pltdata_hybrid = ds_hybrid.sel(ensemble=41, time="1970-01-02T04:00:00")

In [ ]:
figsize = (plt.rcParams["figure.figsize"][0]*0.4, plt.rcParams["figure.figsize"][0]*0.4*50/32)
fig = plt.figure(figsize=figsize, dpi=300)

gridspec = mpl_gs.GridSpec(nrows=1, ncols=32, wspace=50)
ax_hr = fig.add_subplot(gridspec[:, :10])
ax_lr = fig.add_subplot(gridspec[:, 10:20])
ax_hybrid = fig.add_subplot(gridspec[:, 20:30])

ax_hr.axis("off")
ax_lr.axis("off")
ax_hybrid.axis("off")

plt_hr = ax_hr.tripcolor(pltdata_hr.sinn.triangulation, 1-pltdata_hr["damage"].sel(lead_time="1 hour"), cmap="cmo.ice_r", vmin=0., vmax=1, rasterized=True)
plt_fcst = ax_lr.tripcolor(pltdata_forecast.sinn.triangulation, 1-pltdata_forecast["damage"].sel(lead_time="1 hour"), cmap="cmo.ice_r", vmin=0., vmax=1, rasterized=True)
plt_fcst = ax_hybrid.tripcolor(pltdata_hybrid.sinn.triangulation, 1-pltdata_hybrid["damage"].sel(lead_time="1 hour"), cmap="cmo.ice_r", vmin=0., vmax=1, rasterized=True)

ax_hr.set_xlim(-20000, 20000)
ax_hr.set_ylim(-100000, 100000)

ax_lr.set_xlim(-20000, 20000)
ax_lr.set_ylim(-100000, 100000)

ax_hybrid.set_xlim(-20000, 20000)
ax_hybrid.set_ylim(-100000, 100000)

ax_hr.text(x=0.5, y=1.01, s="(a) High-res", transform=ax_hr.transAxes, ha="center", va="bottom", fontsize=9)
ax_lr.text(x=0.5, y=1.01, s="(b) Low-res", transform=ax_lr.transAxes, ha="center", va="bottom", fontsize=9)
ax_hybrid.text(x=0.5, y=1.01, s="(c) Hybrid", transform=ax_hybrid.transAxes, ha="center", va="bottom", fontsize=9)

ax_cbar = fig.add_subplot(gridspec[:, 30:])
cbar = fig.colorbar(plt_fcst, cax=ax_cbar)
cbar.set_label("Damage (1)")

fig.savefig("figures/fig01_intro_multifractality.pdf", pad_inches=0., bbox_inches="tight")